In [67]:
import pandas as pd
import sqlite3
con = sqlite3.connect("database.sqlite")

def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """
    query_output = pd.read_sql_query(query, con)
    return pd.DataFrame(query_output)


In [68]:
# Load the data into a DataFrame
df_country = show_query("SELECT * FROM Country")
df_league = show_query("SELECT * FROM League")
df_match = show_query("SELECT * FROM Match")
df_player = show_query("SELECT * FROM Player")
df_player_attributes = show_query("SELECT * FROM Player_Attributes")
df_team = show_query("SELECT * FROM Team")
df_team_attributes = show_query("SELECT * FROM Team_Attributes")

Maak een functie waarmee je per seizoen voor een gekozen club o.a. het volgende kunt bepalen:

- Aantal wedstrijden in dat seizoen.
- Aantal punten behaald in dat seizoen.
- Gemiddeld aantal punten per wedstrijd.
- Winst-, verlies- en gelijk percentage in uitwedstrijden en thuiswedstrijden.

In [101]:
def details_season(season, club_id):
    """Docstrings
    
    """
    df_home = df_match[(df_match['season'] == season) & (df_match['home_team_api_id'] == club_id)] 
    df_away = df_match[(df_match['season'] == season) & (df_match['away_team_api_id'] == club_id)]
    
    home_points = 0
    for index, rows in df_home.iterrows():
        if rows['home_team_goal'] > rows['away_team_goal']:
            home_points += 3
        elif rows['home_team_goal'] < rows['away_team_goal']:
            home_points += 0
        else:
            home_points += 1
    
    away_points = 0
    for index, rows in df_away.iterrows():
        if rows['away_team_goal'] > rows['home_team_goal']:
            away_points += 3
        elif rows['away_team_goal'] < rows['home_team_goal']:
            away_points += 0
        else:
            away_points += 1
    
    amount_of_games = len(df_home) + len(df_away)
    amount_of_points = home_points + away_points
        
    print('Amount of games this season: ' + str(amount_of_games))
    print('Amount of points this season: ' + str(amount_of_points))
    
    return details_season
        

In [107]:
details_season('2008/2009', 10228)

Amount of games this season: 34
Amount of points this season: 60


<function __main__.details_season(season, club_id)>